<a href="https://colab.research.google.com/github/je-audit/CortexOne/blob/master/selecPlan2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# cargar librerías y otros archivos
import random
#random.seed(1) #un inicializador de números aleatorios

#0) Se modifica el algoritmo considerando conjuntos aleatorios en vez de sbconjuntos con valores iniciales estáticos
#1) Inicialización de variales
#recorre el primero (extrae el primero y lo agrega al package list)
#si llegamos al fin o al filtro
varianzaMenorFinal=100000 #valor tope de la varianza, un valor aproximado, marca un tope
beneficioListFinal=[] #lista del beneficio final de la solución
inversionListFinal=[] #lista de la inversión final de la solución
varianzaListFinal=[] #menor valor de varianza encontrada en la búsqueda
debug=False #control interno de recorridos-

#Funciones
#1) determina la suma de una lista
def suma(lista):
    sumaTotal = 0
    for x in lista:
        sumaTotal += x
    return sumaTotal
#2) imprime la lista de los recorridos en las búsquedas
def printList(lista, nombre):
    global debug
    if not debug:
        return
    print("lista " + nombre + ":[", end='')
    for x in lista:
        print(str(x), end=',')
    print("]")
#3) Imprime una opción de recorrido de búsqueda
def printEstado(titulo, nextBeneficio, nextInversion, nextVarianza, beneficioList, inversionList, varianzaList, indice):
    global debug
    if not debug:
        return
    print(titulo , end=',')
    print("Indice:"+ str(indice) , end=',')

    print("nextBeneficio:"+ str(nextBeneficio) , end=',')
    print("nextInversion:"+ str(nextInversion) , end=',')
    print("nextVarianza:"+ str(nextVarianza) , end=',')
   
    beneficioActual = suma(beneficioList)
    print("Beneficio Actual:"+ str(beneficioActual), end=',' )

    inversionActual = suma(inversionList)
    print("Inversion Actual:"+ str(inversionActual), end=',' )

    varianzaActual = suma(varianzaList)
    print("Varianza Actual:"+ str(varianzaActual) )

#5) Evalúa mejores opciones que cumplan con las restricciones planteadas: beneficio >300 inversión máxima < 3.600
def siguiente(beneficioInicial, inversionInicial, varianzaInicial, beneficioListInicial, inversionListInicial, varianzaListInicial, indice):
    beneficio = beneficioInicial.copy()
    inversion = inversionInicial.copy()
    varianza = varianzaInicial.copy()
    beneficioList = beneficioListInicial.copy()
    inversionList = inversionListInicial.copy()
    varianzaList = varianzaListInicial.copy()
    global beneficioMinimoDeseado
    global inversionMaximaPosible

    printEstado("Estado Inicial:", beneficio[indice], inversion[indice], varianza[indice], beneficioList, inversionList, varianzaList, indice)
   
    if ((suma(inversionList) + inversion[indice]) < inversionMaximaPosible):
        beneficioList.append(beneficio.pop(indice))
        inversionList.append(inversion.pop(indice))
        varianzaList.append(varianza.pop(indice))
    else:
        #No explora más esta rama, supera la restricción de inversión
        return
    if ( suma(inversionList) < inversionMaximaPosible and  suma(beneficioList) > beneficioMinimoDeseado ) :
        print ("----------------Lista Encontrada:------------------")
        printList(beneficioList, "Beneficio")
        printList(inversionList, "Inversion")
        printList(varianzaList, "Varianza")
        print("Varianza Total Encontrada:"+str(suma(varianzaList)))
       
        global varianzaMenorFinal
        global beneficioListFinal
        global inversionListFinal
        global varianzaListFinal
        print("Varianza Final Actual:"+str(varianzaMenorFinal))
        if(suma(varianzaList) < varianzaMenorFinal):
            print("Reemplazo todas las listas finales")
            beneficioListFinal = beneficioList.copy()
            inversionListFinal = inversionList.copy()
            varianzaListFinal = varianzaList.copy()
            varianzaMenorFinal = suma(varianzaList)
        print ("---------------------------------------------------")
    i = 0
    varLoop = 0
    while i < len(beneficio):
        varLoop = 1
        printList(beneficioList, "Beneficio")
        printList(beneficio, "Original")
        if len(beneficio) > 0:
            siguiente(beneficio, inversion, varianza, beneficioList, inversionList, varianzaList, i)
        i += 1
    printEstado("Fin While", beneficioInicial[indice], inversionInicial[indice], varianzaInicial[indice], beneficioListInicial, inversionListInicial, varianzaListInicial, indice)
    if (i == len(beneficio) and varLoop == 1 ):
        beneficioList = beneficioListInicial.copy()
        inversionList = inversionListInicial.copy()
        varianzaList = varianzaListInicial.copy()
        beneficio = beneficioInicial.copy()
        inversion = inversionInicial.copy()
        varianza = varianzaInicial.copy()
        printList(beneficioList, "BeneficioDespuesControl")
        printList(beneficio, "OriginalDespuesControl")


numItems=20 #número de paquetes de inversión
beneficio=[65,36,30,97,37,58,31,97,61,84,62,34,44,84,50,78,32,96,93,69] #beneficio por inversión. Es un vector de 20 posiciones
inversion=[583,600,504,493,503,597,544,403,588,500,526,543,469,541,591,434,586,581,538,517]
varianza=[603,259,128,1045,456,560,137,1045,1240,705,480,144,242,705,500,760,170,1152,864,680] #varianza de la inversión. Es un vector de 20 posiciones
#la matriz de covarianza es simétrica. Se presenta completa por sencillez. Es una matriz de 20x20
covarianza=[[0, -137, -56, 335, 19, 139, 120, -180, 193, 52, 114, -41, -45, 46, -60, 184, 69, -20, 81, 7],
            [-137, 0, -49, -146, 41, 78, 120, 134, -205, -151, 254, 13, -50, 59, 247, 84, 159, 120, -27, 198],
            [-56, -49, 0, 30, -92, -52, 34, 33, -54, -60, 7, -21, 43, -27, -123, 36, 28, -16, 116, 34],
            [335, -146, 30, 0, 203, -189, 10, 863, 514, 490, 358, -144, 10, 389, 248, -249, 88, 972, 603, 149],
            [19, 41, -92, 203, 0, -28, -12, 160, -174, 448, 168, 51, 6, -19, 5, 106, 84, 17, 308, 339],
            [139, 78, -52, -189, -28, 0, 113, -30, 498, -64, 133, -85, 38, 135, -111, -124, 99, 318, -166, 510],
            [120, 120, 34, 10, -12, 113, 0, -191, 51, -98, 115, 122, 21, 133, -108, -17, 66, 87, 30, 4],
            [-180, 134, 33, 863, 160, -30, -191, 0, 591, 21, -240, -163, -205, -49, 35, -9, 143, 28, 686, 507],
            [193, -205, -54, 514, -174, 498, 51, 591, 0, -232, -238, 40, -21, 220, 472, 488, -184, -194, 194, 510],
            [52, -151, -60, 490, 448, -64, -98, 21, -232, 0, 85, 32, 178, 422, 63, 659, 34, 218, 199, 221],
            [114, 254, 7, 358, 168, 133, 115, -240, -238, 85, 0, -103, -33, -46, 263, 22, -57, 453, -210, 380],
            [-41, 13, -21, -144, 51, -85, 122, -163, 40, 32, -103, 0, 75, -37, 127, -11, -35, -280, 42, -154],
            [-45, -50, 43, 10, 6, 38, 21, -205, -21, 178, -33, 75, 0, -137, -13, 226, 27, 180, 12, -115],
            [46, 59, -27, 389, -19, 135, 133, -49, 220, 422, -46, -37, -137, 0, 80, 343, -99, 0, -160, 224],
            [-60, 247, -123, 248, 5, -111, -108, 35, 472, 63, 263, 127, -13, 80, 0, -180, -19, 320, 129, 238],
            [184, 84, 36, -249, 106, -124, -17, -9, 488, 659, 22, -11, 226, 343, -180, 0, 151, -105, -127, 602],
            [69, 159, 28, 88, 84, 99, 66, 143, -184, 34, -57, -35, 27, -99, -19, 151, 0, 79, 98, 38],
            [-20, 120, -16, 972, 17, 318, 87, 28, -194, 218, 453, -280, 180, 0, 320, -105, 79, 0, 71, 652],
            [81, -27, 116, 603, 308, -166, 30, 686, 194, 199, -210, 42, 12, -160, 129, -127, 98, 71, 0, 459],
            [7, 198, 34, 149, 339, 510, 4, 507, 510, 221, 380, -154, -115, 224, 238, 602, 38, 652, 459, 0]]

beneficioMinimoDeseado=300
inversionMaximaPosible=3600
cantidadElementos = 10
#beneficioMinimoDeseado=160 #controles internos con valores menores
#inversionMaximaPosible=1600 #controles con valores menores

#beneficioTest=[1,2,3,4,5]
#beneficioTestAleatorio=[65,36,30,97,37] #set de datos iniciales con 5 nodos de la población objetivo
#inversionTestAleatorio=[583,600,504,493,503] #set de datos iniciales con 5 nodos de la población objetivo
#varianzaTestAleatorio=[603,259,128,1045,456] #set de datos iniciales con 5 nodos de la población objetivo
#generación de subconjunto de valores de manera aleatoria
beneficioTestAleatorio = []
inversionTestAleatorio = []
varianzaTestAleatorio = []

j = 0
while j < cantidadElementos:
    numeroRandom = random.randint(0,9)  
    beneficioTestAleatorio.append(beneficio[numeroRandom])
    inversionTestAleatorio.append(inversion[numeroRandom])
    varianzaTestAleatorio.append(varianza[numeroRandom])
    j = j+1
print("Subconjunto de 5 elementos generado de manera aleatoria en lista de beneficio", beneficioTestAleatorio)
print("Subconjunto de 5 elementos generado de manera aleatoria en la lista de inversión", inversionTestAleatorio)
print("Subconjunto de 5 elementos generado de manera aleatoria en la lista de varianza", varianzaTestAleatorio)


beneficioList=[] #lista de recorrido de beneficios
inversionList=[] #lista de recorrido de inversión
varianzaList=[] #lista de recorrido de varianzas

i = 0
while i < len(beneficioTestAleatorio):#pruebas para un set de datos de 10 nodos
    varLoop = 1
    printList(beneficioList, "Beneficio")
    #printList(beneficioTest10, "Original")
    printList(beneficioTestAleatorio, "Original")
    if len(beneficioTestAleatorio) > 0:
        #siguiente(beneficioTest10, inversionTest10, varianzaTest10, beneficioList, inversionList, varianzaList, i)
        siguiente(beneficioTestAleatorio, inversionTestAleatorio, varianzaTestAleatorio, beneficioList, inversionList, varianzaList, i) #valores para 5 nodos
        #siguiente(beneficio, inversion, varianza, beneficioList, inversionList, varianzaList, i) #control
    i += 1

debug = True
printList(beneficioListFinal, "=====>Lista Final de Beneficio")
printList(inversionListFinal, "=====>Lista Final de Inversión")
printList(varianzaListFinal, "=====>Lista Final de Varianza")
print("Menor Varianza Final Encontrada:"+str(varianzaMenorFinal))

Se han truncado las últimas 5000 líneas del flujo de salida.
Varianza Final Actual:2680
---------------------------------------------------
----------------Lista Encontrada:------------------
Varianza Total Encontrada:4032
Varianza Final Actual:2680
---------------------------------------------------
----------------Lista Encontrada:------------------
Varianza Total Encontrada:4018
Varianza Final Actual:2680
---------------------------------------------------
----------------Lista Encontrada:------------------
Varianza Total Encontrada:4578
Varianza Final Actual:2680
---------------------------------------------------
----------------Lista Encontrada:------------------
Varianza Total Encontrada:4146
Varianza Final Actual:2680
---------------------------------------------------
----------------Lista Encontrada:------------------
Varianza Total Encontrada:4602
Varianza Final Actual:2680
---------------------------------------------------
----------------Lista Encontrada:-----------------

KeyboardInterrupt: ignored